# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [15]:
# import findspark
# findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# May take awhile locally
spark = SparkSession.builder.appName("aggregate").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in the dataFrame for this Notebook

In [2]:
airbnb = spark.read.csv("Datasets/nyc_air_bnb.csv", inferSchema=True, header=True)

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking. 

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [3]:
airbnb.limit(4).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194


Now print the schema so we can make sure all the variables have the correct types

In [4]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [5]:
from pyspark.sql.types import IntegerType

df = (
    airbnb.withColumn("price", airbnb["price"].cast(IntegerType()))
    .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType()))
    .withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType()))
    .withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType()))
    .withColumn(
        "calculated_host_listings_count",
        airbnb["calculated_host_listings_count"].cast(IntegerType()),
    )
)
# QA
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [6]:
df.count()

49079

### 2. How many total reviews does each host have?

In [11]:
df.groupBy("host_id").sum("number_of_reviews").toPandas()

,host_id,sum(number_of_reviews)
0,716306,197.0
1,1203500,35.0
2,368528,1.0
3,1577493,16.0
4,1390555,50.0
...,...,...
37480,181773256,0.0
37481,73986440,1.0
37482,2967377,0.0
37483,271015294,0.0


### 3. Show the min and max of all the numeric variables in the dataset

In [14]:
df.select(
    "price",
    "minimum_nights",
    "number_of_reviews",
    "last_review",
    "reviews_per_month",
    "calculated_host_listings_count",
    "availability_365",
).summary("min", "max").toPandas()

,summary,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,min,-74,0,0,-73.94134,0,0,0
1,max,10000,1250,629,9.66,58,365,365


### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [19]:
df.groupBy("host_id").agg(F.sum("number_of_reviews").alias("Sum Reviews")).orderBy(F.desc("Sum Reviews")).limit(1).toPandas()

,host_id,Sum Reviews
0,37312959,2273


### 5. On average, how many nights did most hosts specify for a minimum?

In [22]:
df.agg(F.mean("minimum_nights")).show()

+-------------------+
|avg(minimum_nights)|
+-------------------+
| 7.1286126280910596|
+-------------------+



### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [23]:
df.groupBy("neighbourhood").agg(F.avg("price").alias("Avg_Price")).orderBy(F.desc("Avg_Price")).limit(1).toPandas()

,neighbourhood,Avg_Price
0,Fort Wadsworth,800.0


### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [25]:
df.filter("room_type='Shared room' OR room_type='Private room'").groupBy("room_type").pivot(
    "neighbourhood_group", ["Manhattan", "Brooklyn"]
).avg("Price").show(100)

+------------+------------------+-----------------+
|   room_type|         Manhattan|         Brooklyn|
+------------+------------------+-----------------+
| Shared room| 89.06903765690376|50.52784503631961|
|Private room|116.05400302114803|76.47234042553191|
+------------+------------------+-----------------+



### Alright that's all folks!

### Great job!